In [ ]:
# ==== Scanning Functions ====
def scanPort(target, port, protocol='TCP'):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM if protocol == 'TCP' else socket.SOCK_DGRAM)
        s.settimeout(2)
        if protocol == 'TCP':
            c = s.connect_ex((target, port))
            if c == 0:
                m = f' Port {port} \t[open] ({protocol})'
                log.append(m)
                ports.append(port)
                listbox.insert("end", str(m))
                checkVulnerability(port)
                updateResult()
        else:  # UDP scanning
            try:
                s.sendto(b"", (target, port))
                s.recvfrom(1024)
                m = f' Port {port} \t[open] ({protocol})'
                log.append(m)
                ports.append(port)
                listbox.insert("end", str(m))
                checkVulnerability(port)
                updateResult()
            except socket.error:
                pass
        s.close()
    except OSError:
        print(f'> Too many open sockets. Port {port}')
    except Exception as e:
        print(f'> Error on port {port}: {e}')
        sys.exit()
    sys.exit()

def checkVulnerability(port):
    if port in vuln_db:
        vuln_msg = f" Vulnerability: {vuln_db[port]}"
        mitigation_msg = f" Mitigation: {mitigation_db.get(port, 'No mitigation available.')}"
        log.append(vuln_msg + " " + mitigation_msg)
        listbox.insert("end", f"{vuln_msg} | {mitigation_msg}")

def updateResult():
    rtext = f" [ {len(ports)} / {ip_f} ] ~ {target} ({protocol})"
    L27.configure(text=rtext)

def startScan():
    global ports, log, target, ip_f, protocol
    clearScan()
    log = []
    ports = []
    ip_s = int(L24.get())
    ip_f = int(L25.get())
    protocol = protocol_var.get()
    log.append(f'> Port Scanner ({protocol})')
    log.append('=' * 14 + '\n')
    log.append(' Target:\t' + str(target))

    try:
        target = socket.gethostbyname(str(L22.get()))
        log.append(' IP Adr.:\t' + str(target))
        log.append(f' Ports: \t[ {ip_s} / {ip_f} ]')
        log.append('\n')
        
        while ip_s <= ip_f:
            try:
                scan = threading.Thread(target=scanPort, args=(target, ip_s, protocol))
                scan.setDaemon(True)
                scan.start()
            except:
                time.sleep(0.01)
            ip_s += 1
    except:
        m = f'> Target {L22.get()} not found.'
        log.append(m)
        listbox.insert(0, str(m))

def saveScan():
    global log, target, ports, ip_f
    log[5] = f" Result:\t[ {len(ports)} / {ip_f} ]\n"
    with open(f'portscan-{target}.txt', mode='wt', encoding='utf-8') as myfile:
        myfile.write('\n'.join(log))

def clearScan():
    listbox.delete(0, 'end')